In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv("../datasets/titanic/train.csv")

In [3]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [4]:
cut = df[['Survived', 'Pclass', 'Sex', 'Age', 'SibSp',
       'Parch']].copy()

In [5]:
cut.describe()

,Survived,Pclass,Age,SibSp,Parch
count,891.000000,891.000000,714.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594
std,0.486592,0.836071,14.526497,1.102743,0.806057
min,0.000000,1.000000,0.420000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000
50%,0.000000,3.000000,28.000000,0.000000,0.000000
75%,1.000000,3.000000,38.000000,1.000000,0.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000


In [6]:
cut.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
dtypes: float64(1), int64(4), object(1)
memory usage: 41.8+ KB


In [7]:
droped = cut.loc[cut["Age"].isna()]

In [8]:
cut.dropna(inplace=True)

In [9]:
board = cut.groupby(["Survived", "Pclass", "Sex"])[["Age"]].mean()
board.reset_index(inplace=True)

In [24]:
board

,Survived,Pclass,Sex,Age
0,0,1,female,25.666667
1,0,1,male,44.581967
2,0,2,female,36.000000
3,0,2,male,33.369048
4,0,3,female,23.818182
5,0,3,male,27.255814
6,1,1,female,34.939024
7,1,1,male,36.248000
8,1,2,female,28.080882
9,1,2,male,16.022000


In [10]:
def get_age(row):
    item = board.loc[(board["Survived"]==row["Survived"]) & (board["Pclass"]==row["Pclass"]) & (board["Sex"]==row["Sex"])]
    row["Age"] = item.iat[0,-1]
    return row

In [11]:
board.loc[(board["Survived"]==0) & (board["Pclass"]==1) & (board["Sex"]=="male")]

,Survived,Pclass,Sex,Age
1,0,1,male,44.581967


In [12]:
droped = droped.apply(get_age, axis=1)

In [13]:
total = pd.concat([cut, droped])

In [14]:
total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 888
Data columns (total 6 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
dtypes: float64(1), int64(4), object(1)
memory usage: 48.7+ KB


In [31]:
le = LabelEncoder()

In [32]:
total["Sex"] = le.fit_transform(total["Sex"])

In [33]:
le.classes_

array([0, 1])

In [34]:
X = total.drop(["Survived"], axis=1)

In [35]:
y = total["Survived"]

In [36]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.33, random_state=42)

In [69]:
model = DecisionTreeClassifier(max_depth=5)

In [70]:
model.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [71]:
model.score(X_test, y_test)

0.8338983050847457

In [27]:
def entropy(col):
    vals, counts = np.unique(col, return_counts=True)
    
    acc = 0
    probs = counts / len(col)
    for prob in probs:
        acc += prob * np.log2(prob)
        
    return - acc

In [28]:
entropy(total["Survived"])

0.9607079018756469

In [29]:
def infogain(X, col_name, y):
    
    pivot = X[col_name].mean()
    left = y.loc[X[col_name] < pivot]
    right = y.loc[X[col_name] >= pivot]
    
    l_entropy = entropy(left)
    r_entropy = entropy(right)
    tot_entropy = entropy(y)
    p_left = len(left)/len(y)
    p_right = len(right)/len(y)
    
    return tot_entropy - p_left*l_entropy - p_right*r_entropy


In [30]:
for col in X.columns:
    print("infogain for ", col, infogain(X, col, y))

infogain for  Pclass 0.07579362743608165
infogain for  Sex 0.2176601066606143
infogain for  Age 0.0008052404301705507
infogain for  SibSp 0.009584541813400127
infogain for  Parch 0.015380754493137666


In [157]:
class Node:
    
    def __init__(self, label=None, pivot=None, result=None):
        
        self.label = label
        self.pivot = pivot
        self.result = result
        
        self.left = None
        self.right = None
    
class DecisionTreeCuston:
    
    def __init__(self, max_depth=5):
        self.max_depth = max_depth
        self.nodes = 0
        
    def fit(self, X, y):
        self.root = self.generate(X, y, self.max_depth)
    
    def generate(self, X, y, depth):
        if depth == 0:
            node = Node(result=y.mean())
            self.nodes += 1
            return node
        
        gains = []
        for col in X.columns:
            gains.append((self.infogain(X, col, y), col))
            
        selected = sorted(gains, reverse=True)[0]
        selected_gain = selected[0]
        selected_col = selected[1]
        
        if selected_gain <= .01:
            node = Node(result=y.mean())
            self.nodes += 1
            return node
        
        pivot = X[selected_col].mean()
        
        left_X = X.loc[X[selected_col] < pivot]
        right_X = X.loc[X[selected_col] >= pivot]
        
        left_y = y.loc[X[selected_col] < pivot]
        right_y = y.loc[X[selected_col] >= pivot]
        
        node = Node(label=selected_col, pivot=pivot)
        node.left = self.generate(left_X, left_y, depth-1)
        node.right = self.generate(right_X, right_y, depth-1)
        self.nodes += 1
        
        return node
    
    def predict_point(self, row, node):
        if node.label == None:
            return node.result
        
        if row[node.label] < node.pivot:
            return self.predict_point(row, node.left)
        else:
            return self.predict_point(row, node.right)
    
    def predict(self, X):
        results = []
        for index, row in X.iterrows():
            results.append(self.predict_point(row, self.root))
            
        return (np.array(results) > .7).astype(int)
            
    
    def entropy(self, col):
        vals, counts = np.unique(col, return_counts=True)
        acc = 0
        probs = counts / len(col)
        for prob in probs:
            acc += prob * np.log2(prob)
        return - acc
    
    def score(self, X, y):
        y_p = self.predict(X)
        return (y == y_p).mean()
    
    def display(self):
        self.display_rec(self.root, "")
    
    def display_rec(self, node, indent):
        if node == None:
            return
        
        if node.label != None:
            print(indent, "Deviding abt", node.label, node.pivot)
            self.display_rec(node.left, indent + "\t")
            self.display_rec(node.right, indent + "\t")
        else:
            if node.result > .5:
                print(indent, "Survived")
            else:
                print(indent, "Died")
    
    def infogain(self, X, col_name, y):
    
        pivot = X[col_name].mean()
        left = y.loc[X[col_name] < pivot]
        right = y.loc[X[col_name] >= pivot]

        l_entropy = entropy(left)
        r_entropy = entropy(right)
        tot_entropy = entropy(y)
        p_left = len(left)/len(y)
        p_right = len(right)/len(y)

        return tot_entropy - p_left*l_entropy - p_right*r_entropy

        
        

In [158]:
model = DecisionTreeCuston(max_depth=10)

In [159]:
model.fit(X_train, y_train)

In [160]:
model.predict(X_test.iloc[:10])

array([0, 0, 0, 1, 1, 0, 0, 1, 1, 0])

In [161]:
model.nodes

105

In [162]:
model.score(X_test, y_test)

0.8271186440677966

In [142]:
model.display()

 Deviding abt Sex 0.6593959731543624
	 Deviding abt Pclass 2.147783251231527
		 Survived
		 Deviding abt Age 21.4670451945675
			 Deviding abt Age 14.062340425531916
				 Deviding abt Age 5.264705882352941
					 Deviding abt Age 2.772727272727273
						 Deviding abt Age 1.4166666666666667
							 Survived
							 Died
						 Survived
					 Died
				 Deviding abt Age 18.594455190199874
					 Deviding abt Age 16.75
						 Survived
						 Deviding abt Parch 0.5714285714285714
							 Died
							 Deviding abt SibSp 1.3333333333333333
								 Died
								 Survived
					 Deviding abt Age 19.6484295845998
						 Survived
						 Deviding abt SibSp 0.8
							 Died
							 Died
			 Deviding abt SibSp 1.0930232558139534
				 Died
				 Died
	 Deviding abt Pclass 2.3587786259541983
		 Deviding abt Pclass 1.4819277108433735
			 Deviding abt Age 41.93866946244758
				 Deviding abt Age 30.29368421052632
					 Deviding abt Age 22.6825
						 Deviding abt Parch 1.5
							 Died
							 Survived
				

In [143]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, 

In [144]:
RandomForestClassifier?

Init signature:
RandomForestClassifier(
    ["n_estimators='warn'", "criterion='gini'", 'max_depth=None', 'min_samples_split=2', 'min_samples_leaf=1', 'min_weight_fraction_leaf=0.0', "max_features='auto'", 'max_leaf_nodes=None', 'min_impurity_decrease=0.0', 'min_impurity_split=None', 'bootstrap=True', 'oob_score=False', 'n_jobs=None', 'random_state=None', 'verbose=0', 'warm_start=False', 'class_weight=None'],
)
Docstring:     
A random forest classifier.

A random forest is a meta estimator that fits a number of decision tree
classifiers on various sub-samples of the dataset and uses averaging to
improve the predictive accuracy and control over-fitting.
The sub-sample size is always the same as the original
input sample size but the samples are drawn with replacement if
`bootstrap=True` (default).

Read more in the :ref:`User Guide <forest>`.

Parameters
----------
n_estimators : integer, optional (default=10)
    The number of trees in the forest.

    .. versionchanged:: 0.20
       